neural networks

shot_distance
shot_angle
x,y coodinates
royal_road
relative_rest
last_event_type
number_of_shots
shot_type
shooter_left_right
relative_score

power_play
empty_net
predicting_rare_events

In [22]:
def print_results(y_test, y_pred):
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    print("Neural Network Results:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))  

In [23]:
def run_neural_network(working_df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras import regularizers

    X = working_df[['shot_distance', 'shot_type_id','prior_event_id','rest_difference','shooter_time_on_ice','on_man_advantage']]
    y = working_df['is_goal']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    class_weights = {0: 0.07, 1: 0.93}  

    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=6, kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), class_weight=class_weights)

    predictions = model.predict(X_test_scaled)
    y_pred = np.round(predictions).flatten()
    return (y_test, y_pred)


In [19]:
def get_working_dataframe(df_base):
    def shooter_on_man_advantage(awaySkatersOnIce, homeSkatersOnIce,isHomeTeam):
        if isHomeTeam:
            if homeSkatersOnIce > awaySkatersOnIce:
                return True
            else:
                return False
        else:
            if awaySkatersOnIce > homeSkatersOnIce:
                return True
            else:
                return False

    working_df = df_base[['arenaAdjustedShotDistance', 'goal', 'shotType','lastEventCategory','averageRestDifference','shooterTimeOnIce','awaySkatersOnIce', 'homeSkatersOnIce','isHomeTeam']]
    working_df = working_df.iloc[:]
    working_df['shotType'] = working_df['shotType'].fillna('UNK')
    shot_type_mapping = {'WRIST': 1, 'SLAP': 2, 'TIP': 3, 'SNAP': 4, 'BACK': 5, 'WRAP': 6, 'DEFL': 7, 'UNK': 8}
    working_df['shot_type_id'] = working_df['shotType'].map(shot_type_mapping)
    prior_event_mapping = {'HIT': 0, 'FAC': 1, 'BLOCK': 2, 'SHOT': 3, 'GIVE': 4, 'MISS': 5, 'TAKE': 6, 'DELPEN': 7, 'STOP': 8, 'CHL': 9}
    working_df['prior_event_id'] = working_df['lastEventCategory'].map(prior_event_mapping)
    working_df['on_man_advantage'] = working_df.apply(lambda row: shooter_on_man_advantage(row['awaySkatersOnIce'], row['homeSkatersOnIce'], row['isHomeTeam']), axis=1)
    working_df = working_df.rename(columns={'arenaAdjustedShotDistance': 'shot_distance', 'averageRestDifference': 'rest_difference','shooterTimeOnIce': 'shooter_time_on_ice', 'goal': 'is_goal'})
    working_df = working_df[['shot_distance', 'shot_type_id', 'prior_event_id', 'rest_difference','shooter_time_on_ice','on_man_advantage', 'is_goal',]]
    return working_df

In [20]:
def get_base_dataframe():
    import pandas as pd
    file_path = 'shots_2022.csv'
    df_base = pd.read_csv(file_path)
    return df_base

In [24]:
base_dataframe = get_base_dataframe()
working_dataframe = get_working_dataframe(base_dataframe)
(y_test, y_pred) = run_neural_network(working_dataframe)
print_results(y_test, y_pred)

Epoch 1/10
3051/3051 [==============================] - 8s 2ms/step - loss: 0.1530 - accuracy: 0.5342 - val_loss: 0.6638 - val_accuracy: 0.5488
Epoch 2/10
3051/3051 [==============================] - 7s 2ms/step - loss: 0.0870 - accuracy: 0.5622 - val_loss: 0.6836 - val_accuracy: 0.5235
Epoch 3/10
3051/3051 [==============================] - 7s 2ms/step - loss: 0.0867 - accuracy: 0.5589 - val_loss: 0.6777 - val_accuracy: 0.5300
Epoch 4/10
3051/3051 [==============================] - 7s 2ms/step - loss: 0.0863 - accuracy: 0.5599 - val_loss: 0.6499 - val_accuracy: 0.5803
Epoch 5/10
3051/3051 [==============================] - 8s 2ms/step - loss: 0.0864 - accuracy: 0.5637 - val_loss: 0.6678 - val_accuracy: 0.5470
Epoch 6/10
3051/3051 [==============================] - 8s 3ms/step - loss: 0.0865 - accuracy: 0.5663 - val_loss: 0.6567 - val_accuracy: 0.5540
Epoch 7/10
3051/3051 [==============================] - 8s 3ms/step - loss: 0.0863 - accuracy: 0.5686 - val_loss: 0.6223 - val_accuracy: